In [4]:
# general packages
import numpy as np
import pandas as pd
import datatable as dt

In [7]:
%%time
data = pd.read_csv('features_processed.csv') 

Wall time: 24.6 s


In [9]:
data = data.iloc[:,1:]

In [10]:
data.iloc[:,30:45]

,Conditions,AreaNumber,PuLat,PuLong,create_by_hour,request_by_hour,pick,BookingSource_Internet,BookingSource_Permanent,BookingSource_StratumIVR,...,create_part_of_day_Afternoon,create_part_of_day_Evening,create_part_of_day_Morning,create_part_of_day_Night,request_part_of_day_Afternoon,request_part_of_day_Evening,request_part_of_day_Morning,request_part_of_day_Night,BookingisDispatch_False,BookingisDispatch_True
0,0,177,-33.921022,151.244202,10,11,True,0,0,0,...,0,0,1,0,0,0,1,0,0,1
1,0,111,-33.755931,151.273028,9,9,True,0,0,0,...,0,0,1,0,0,0,1,0,0,1
2,0,92,-33.897201,151.261205,0,0,True,1,0,0,...,0,0,0,1,0,0,0,1,0,1
3,0,182,-33.895716,151.228609,9,9,True,0,0,1,...,0,0,1,0,0,0,1,0,0,1
4,0,4,-33.859737,151.213307,21,21,True,1,0,0,...,0,0,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3299995,0,265,-33.827898,151.126799,20,6,True,1,0,0,...,0,1,0,0,0,0,1,0,1,0
3299996,0,149,-33.712023,151.095004,17,17,True,0,0,0,...,1,0,0,0,1,0,0,0,0,1
3299997,0,219,-33.850450,151.134428,8,8,True,0,0,1,...,0,0,1,0,0,0,1,0,0,1
3299998,0,177,-33.915317,151.251008,8,8,True,0,0,0,...,0,0,1,0,0,0,1,0,1,0


In [11]:
data.dtypes.head(10)

Conditions                    int64
AreaNumber                    int32
PuLat                       float64
PuLong                      float64
create_by_hour                int32
request_by_hour               int32
pick                           bool
BookingSource_Internet         bool
BookingSource_Permanent        bool
BookingSource_StratumIVR       bool
dtype: object

## Training

In [7]:
training_data = data.iloc[:3000000,:]
validation_data = data.iloc[3000000:,:]

In [8]:
y_train = training_data[['TargetVariable',]]
X_train = training_data.drop(columns=['TargetVariable'])
y_val = validation_data['TargetVariable']
X_val = validation_data.drop(columns=['TargetVariable'])

KeyError: 'TargetVariable'